In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df=application_df.drop(columns=['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
app_counts=application_df['APPLICATION_TYPE'].value_counts()

# uses the variable name `application_types_to_replace`
max_app_counts = 528

application_types_to_replace = app_counts[app_counts<max_app_counts].index.tolist()
# Replaces in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
class_counts = application_df['CLASSIFICATION'].value_counts()
# uses the variable name `classifications_to_replace`
max_classcount = 1883
classifications_to_replace = class_counts[class_counts<max_classcount].index.tolist()
# Replaces in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Converts categorical data to numeric with `pd.get_dummies`
dummy_cats = ['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS']
dummies_df=pd.get_dummies(application_df, columns=dummy_cats)

dummies_df

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,5000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,5000,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:

# Splits  preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL'].values
X = dummies_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)



In [7]:
# Creates a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [20]:
# OPTIMIZATION ATTEMPT 1: 'Added Neurons to hidden layer'
# using the neuron rule of thumb, but making it three times the amount of features after numericizing the categorical data (132 neurons).

# Defines the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer_1= 3*number_input_features
hidden_nodes_layer_2 = 3*number_input_features
hidden_nodes_layer_3 = 3*number_input_features

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer_1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer_2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 132)               5940      
                                                                 
 dense_9 (Dense)             (None, 132)               17556     
                                                                 
 dense_10 (Dense)            (None, 1)                 133       
                                                                 
Total params: 23,629
Trainable params: 23,629
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
fit_model = nn1.fit(X_train_scaled,y_train,epochs=100, validation_data=(X_test_scaled, y_test), verbose=1)

Epoch 1/100
804/804 [==============================] - 5s 5ms/step - loss: 0.5709 - accuracy: 0.7228 - val_loss: 0.5637 - val_accuracy: 0.7201
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5557 - accuracy: 0.7296 - val_loss: 0.5633 - val_accuracy: 0.7223
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5522 - accuracy: 0.7322 - val_loss: 0.5594 - val_accuracy: 0.7234
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5501 - accuracy: 0.7327 - val_loss: 0.5613 - val_accuracy: 0.7248
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5484 - accuracy: 0.7340 - val_loss: 0.5544 - val_accuracy: 0.7245
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5478 - accuracy: 0.7348 - val_loss: 0.5565 - val_accuracy: 0.7217
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5469 - accuracy: 0.7346 - val_loss: 0.5577 - val_accuracy: 0.7242

In [27]:
# Evaluates Optimization model 1 using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5730 - accuracy: 0.7235 - 422ms/epoch - 2ms/step
Loss: 0.5730211138725281, Accuracy: 0.723498523235321


In [28]:
# Saves Optimization model 1 to HDF5
nn1.save('AlphabetSoupCharity_Optimisation1.h5')

In [41]:
# OPTIMIZATION ATTEMPT #2 REDUCED NEURONS + ADDED LAYER
## Adding fourth layer (1 input, 2 hidden, 1 output) consisting of 27 neurons.

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=27, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=27, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=27, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn2.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 27)                1215      
                                                                 
 dense_28 (Dense)            (None, 27)                756       
                                                                 
 dense_29 (Dense)            (None, 27)                756       
                                                                 
 dense_30 (Dense)            (None, 1)                 28        
                                                                 
Total params: 2,755
Trainable params: 2,755
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compiles the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100, validation_data=(X_test_scaled, y_test), verbose=1)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5754 - accuracy: 0.7199 - val_loss: 0.5628 - val_accuracy: 0.7217
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5556 - accuracy: 0.7326 - val_loss: 0.5636 - val_accuracy: 0.7228
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5532 - accuracy: 0.7321 - val_loss: 0.5584 - val_accuracy: 0.7237
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5511 - accuracy: 0.7338 - val_loss: 0.5594 - val_accuracy: 0.7230
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5497 - accuracy: 0.7344 - val_loss: 0.5587 - val_accuracy: 0.7254
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5489 - accuracy: 0.7340 - val_loss: 0.5586 - val_accuracy: 0.7227
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5483 - accuracy: 0.7342 - val_loss: 0.5604 - val_accuracy: 0.7213

In [44]:
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5624 - accuracy: 0.7230 - 369ms/epoch - 1ms/step
Loss: 0.5623911619186401, Accuracy: 0.7230320572853088


In [45]:
nn2.save('AlphabetSoupCharity_Optimisation2.h5')

In [48]:
#OPTIMIZATION MODEL 3 (REDUCED INPUT FEATURES, INCREASED NEURONS, )
reduced_df=application_df.drop(columns=['APPLICATION_TYPE','CLASSIFICATION',"STATUS","SPECIAL_CONSIDERATIONS"])
reduced_dummy_cats = ['AFFILIATION','USE_CASE','ORGANIZATION','INCOME_AMT']
reduced_dummies = pd.get_dummies(reduced_df, columns = reduced_dummy_cats)
reduced_dummies

,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,1,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,5000,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,5000,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,5000,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [50]:
# Splits our preprocessed data into our features and target arrays
y = reduced_dummies['IS_SUCCESSFUL'].values
X = reduced_dummies.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [51]:
len(X_test_scaled[0])

25

In [77]:
# Defines the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer_1= 3*number_input_features
hidden_nodes_layer_2 = 3*number_input_features
hidden_nodes_layer_3 = 3*number_input_features

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=75, input_dim=number_input_features, activation="LeakyReLU")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=75, activation="LeakyReLU"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=75, activation="LeakyReLU"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_70 (Dense)            (None, 75)                1950      
                                                                 
 dense_71 (Dense)            (None, 75)                5700      
                                                                 
 dense_72 (Dense)            (None, 75)                5700      
                                                                 
 dense_73 (Dense)            (None, 1)                 76        
                                                                 
Total params: 13,426
Trainable params: 13,426
Non-trainable params: 0
_________________________________________________________________


In [78]:
# Compiles the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [79]:
fit_model = nn3.fit(X_train_scaled,y_train,epochs=100, validation_data=(X_test_scaled, y_test), verbose=1)

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.6098 - accuracy: 0.6941 - val_loss: 0.6071 - val_accuracy: 0.6915
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6024 - accuracy: 0.6969 - val_loss: 0.6148 - val_accuracy: 0.6931
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6018 - accuracy: 0.6971 - val_loss: 0.6077 - val_accuracy: 0.6918
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6007 - accuracy: 0.6990 - val_loss: 0.6032 - val_accuracy: 0.6945
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6000 - accuracy: 0.6991 - val_loss: 0.6034 - val_accuracy: 0.6939
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5998 - accuracy: 0.6988 - val_loss: 0.6100 - val_accuracy: 0.6880
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6003 - accuracy: 0.6984 - val_loss: 0.6043 - val_accuracy: 0.6907

KeyboardInterrupt: ignored